In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import keras
from tcn import tcn
from sklearn.model_selection import train_test_split


sc = StandardScaler()

Using TensorFlow backend.


In [6]:
dfs = []
for i in range(3,6):
    df = pd.read_pickle("data/df_MB_"+str(i))
    df.drop("sensor", axis=1, inplace=True)
    dfs.append(df)

train = pd.concat(dfs, axis=1)
print(train.shape)
train.columns = [sensor+m for sensor in ["acc_", "gyro_", "mag_"] for m in ["x", "y", "z"]]
#train["y"] = 0

#dfs = []
#for i in range(3,6):
#    df = pd.read_pickle("data/df_not_squats_"+str(i))
#    df.drop("sensor", axis=1, inplace=True)
#    dfs.append(df)

train2 = pd.read_pickle("data/df_negative") #pd.concat(dfs, axis=1)
train2["mag_z"] = 0
train2 = train2.iloc[:4500]
train2.columns = [sensor+m for sensor in ["acc_", "gyro_", "mag_"] for m in ["x", "y", "z"]]
#train2["y"] = 1
print(train2.shape)
train = pd.concat([train,train2], ignore_index=True, axis=0)
print(train.shape)
train.drop("mag_z", axis=1, inplace=True)
train.reset_index(drop=True, inplace=True)
train.fillna(method="pad", inplace=True)
train

(4500, 9)
(4500, 9)
(9000, 9)


,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,mag_x,mag_y
0,1.935,-5.104,0.690,3.069,0.177,0.579,-15.125,58.500
1,-5.784,-7.403,3.266,2.771,-0.469,0.390,-13.813,56.500
2,-0.881,-5.650,5.104,2.677,-0.504,-0.155,-14.000,53.063
3,0.019,-3.792,7.518,2.652,-1.042,-0.089,-12.813,48.250
4,0.163,-2.835,8.389,2.376,-1.106,-0.100,-13.750,41.938
5,-1.025,-2.145,9.318,2.195,-1.017,-0.303,-16.125,34.188
6,0.766,0.124,10.218,1.957,-1.172,-0.411,-17.063,27.188
7,1.398,0.555,10.898,1.005,-0.634,-0.366,-20.938,17.313
8,2.538,2.950,11.837,-0.210,0.196,0.181,-22.500,11.688
9,3.026,4.070,13.819,-1.679,0.627,0.639,-26.250,5.250


In [7]:
train_sc = sc.fit_transform(train.values.astype("float"))
train_sc.shape

(9000, 8)

In [8]:
new_df = pd.DataFrame(train_sc)

In [9]:
df_pos = new_df.iloc[:4500]
df_neg = new_df.iloc[4500:]

In [10]:
pos_samples = []
neg_samples = []
for i in range(len(df_pos)-45):
    pos = df_pos.iloc[i:i+45].values
    neg = df_neg.iloc[i:i+45].values
    pos_samples.append(pos)
    neg_samples.append(neg)

In [11]:
X_pos = np.stack(pos_samples)
X_neg = np.stack(neg_samples)
X_pos.shape, X_neg.shape

((4455, 45, 8), (4455, 45, 8))

In [12]:
X = np.concatenate([X_pos, X_neg], axis=0)
X.shape

(8910, 45, 8)

In [13]:
y = np.zeros(len(X))
y[:4455] = 1

In [7]:
splits = np.array_split(train_sc, len(train)/45)

In [8]:
X = np.stack(splits)

In [9]:
X.shape

(200, 45, 8)

In [10]:
y = np.zeros(len(X))
y[:100] = 1

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [15]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7128, 45, 8), (1782, 45, 8), (7128,), (1782,))

In [16]:
model, param_str = tcn.dilated_tcn(output_slice_index='last', # try 'first'.
                                       num_feat=8,
                                       num_classes=2,
                                       nb_filters=8,
                                       kernel_size=4,
                                       dilatations=[1, 2, 4, 8],
                                       nb_stacks=4,
                                       max_len=45,
                                       activation='norm_relu',
                                       use_skip_connections=False,
                                       return_param_str=True)

#print(f'x_train.shape = {x_train.shape}')
#print(f'y_train.shape = {y_train.shape}')
# Using sparse softmax.
# http://chappers.github.io/web%20micro%20log/2017/01/26/quick-models-in-keras/
model.summary()


x.shape= (?, 8)
model.x = (?, 45, 8)
model.y = (?, 2)
Adam with norm clipping.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 45, 8)        0                                            
__________________________________________________________________________________________________
initial_conv (Conv1D)           (None, 45, 8)        264         input_layer[0][0]                
__________________________________________________________________________________________________
dilated_conv_2_tanh_s0 (Conv1D) (None, 45, 8)        264         initial_conv[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 45, 8)        0           dilated_conv_2_tanh_s0[0][0]     
______________________________

In [18]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), 
          epochs=2,batch_size=64)

Train on 7128 samples, validate on 1782 samples
Epoch 1/2
7128/7128 [==============================] - 4s 607us/step - loss: 0.0254 - acc: 0.9917 - val_loss: 0.0023 - val_acc: 1.0000
Epoch 2/2
7128/7128 [==============================] - 4s 573us/step - loss: 0.0100 - acc: 0.9969 - val_loss: 6.1982e-04 - val_acc: 1.0000


In [20]:
from keras.models import load_model, save_model
import pickle

In [21]:
save_model(model,'squat_detector_3.h5')

In [21]:
model.predict(X_test[17:18])

array([[9.9999714e-01, 2.8914906e-06]], dtype=float32)

In [22]:
pickle.dump(sc, open("data/sc_3.pkl", "wb"))

In [19]:
y_test

array([1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0.,
       1., 0., 0., 0., 0., 0.])

In [24]:
# USAGE
# Start the server:
# 	python run_keras_server.py
# Submit a request via cURL:
# 	curl -X POST -F image=@dog.jpg 'http://localhost:5000/predict'
# Submita a request via Python:
#	python simple_request.py

# import the necessary packages
from keras.applications import ResNet50
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from keras.models import load_model
from PIL import Image
import numpy as np
import flask
import io

# initialize our Flask application and the Keras model
app = flask.Flask(__name__)
model = None



def load_model():
	# load the pre-trained Keras model (here we are using a model
	# pre-trained on ImageNet and provided by Keras, but you can
	# substitute in your own networks just as easily)
	global model
	model = load_model('squat_detector.h5') #ResNet50(weights="imagenet")

    
@app.route("/predict", methods=["POST"])
def predict():
	# initialize the data dictionary that will be returned from the
	# view
	data = {"success": False, "squat_prob":None}

	# ensure an image was properly uploaded to our endpoint
	if flask.request.method == "POST":
		if flask.request.files.get("input"):
			# read the image in PIL format
			x_b = flask.request.files["input"].read()
			x = np.frombuffer(x)
            squat_prob = model.predict(x)[0][1]
			# indicate that the request was a success
            data["squat_prob"] = squat_prob
			data["success"] = True

	# return the data dictionary as a JSON response
	return flask.jsonify(data)

# if this is the main thread of execution first load the model and
# then start the server
if __name__ == "__main__":
	print(("* Loading Keras model and Flask starting server..."
		"please wait until server has fully started"))
	load_model()
	app.run()